# Norvig

In [9]:
import re
from collections import Counter
from pathlib import Path

class Norvig_corrector: # from https://norvig.com/spell-correct.html

    def words(self, text): return re.findall(r'\w+', text.lower())

    def __init__(self):
        self.WORDS = Counter(self.words(" ".join([re.sub(r'[^\x00-\x7F]+',' ', open(str(path)).read().lower().replace('\n', ' ')) for path in Path('../data/scrapped/class_data_function__1_1/').rglob('*.txt')])))

    def P(self, word): 
        N=sum(self.WORDS.values())
        "Probability of `word`."
        return self.WORDS[word] / N

    def correction(self, word): 
        "Most probable spelling correction for word."
        return max(self.candidates(word), key=self.P)

    def candidates(self, word): 
        "Generate possible spelling corrections for word."
        return (self.known([word]) or self.known(self.edits1(word)) or self.known(self.edits2(word)) or set(word,))

    def known(self, words): 
        "The subset of `words` that appear in the dictionary of WORDS."
        return set(w for w in words if w in self.WORDS)

    def edits1(self, word):
        "All edits that are one edit away from `word`."
        letters    = 'abcdefghijklmnopqrstuvwxyz'
        splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
        deletes    = [L + R[1:]               for L, R in splits if R]
        transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
        replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
        inserts    = [L + c + R               for L, R in splits for c in letters]
        return set(deletes + transposes + replaces + inserts)

    def edits2(self, word): 
        "All edits that are two edits away from `word`."
        return (e2 for e1 in self.edits1(word) for e2 in self.edits1(e1))
    
    def fix_typo_norvig(self, word) -> str:
        "Most probable spelling correction for word."
        return max(self.candidates(word), key=self.P)